In [1]:
import itertools as it

In [2]:
# Load all hero's names.
file_path = r"C:\Users\Chris\Documents\GitHub\strevr-data\Dota 2\Heroes\hero_names.txt"
with open(file_path) as fin:
    hero_names = [s.strip().upper() for s in fin]
print(len(hero_names), 'heroes')

# Make the alphabet of characters in all of the hero names.
alphabet = set(it.chain.from_iterable(hero_names))
l = sorted(alphabet)
print(len(l), *l, sep=', ')

# Get the hero names containing a letter.
def get_hero_names_with_letter(letter: str):
    l = [s for s in hero_names if letter in s]
    return l

# Get the list of names that contain letters found in no other names. All methodologies start with this list.
l = [get_hero_names_with_letter(c) for c in alphabet]
unique_names = {l[0] for l in l if len(l) == 1}
print(*sorted(unique_names), sep=', ')

126 heroes
29,  , ', -, A, B, C, D, E, F, G, H, I, J, K, L, M, N, O, P, Q, R, S, T, U, V, W, X, Y, Z
ANTI-MAGE, NATURE'S PROPHET, QUEEN OF PAIN


In [3]:
def collect_necessary_names(select_names):
    l = sorted(select_names())
    print(len(l), len(list(it.chain.from_iterable(l))), *l, sep=', ')

This algorithm starts with the longest name, the shortest name, and the unique names. It constructs the set of letters, the accounted set, from those names and follows these steps.

1. Select the unaccounted letter that is in the fewest names of those in the available names. Resolve ties by selecting the letter whose names encompass the most of the unaccounted letters.
1. From the list of available names that contain that letter, move the name with the greatest number of letters that are not represented in the list of selected names from the list of available names to the list of selected names.
1. Update the set of accounted letters.

In [4]:
def select_names():
    # Create sets of available and selected hero names from the unique names.
    selected_hero_names = set(unique_names)
    available_hero_names = set(hero_names) - selected_hero_names
    # Create a set of accounted letters.
    accounted_letters = set(it.chain.from_iterable(selected_hero_names))
    nletters = len(alphabet)
    while len(accounted_letters) < nletters:
        # Create a list of letters that are not represented in the list of selected names.
        letters = alphabet - accounted_letters
        # From that list, select the letter that is in the fewest of the available
        # names and encompasses the most of the unaccounted letters.
        letter = get_letter(letters, available_hero_names)
        # Get the list of available names that contain that letter.
        names = get_names(letter, available_hero_names)
        # From that list, select the name with the greatest number of letters
        # that are not represented in the list of selected names.
        name = get_name(names, letters)
        # Add that name to the list of selected names.
        selected_hero_names.add(name)
        available_hero_names.remove(name)
        # Add that name's letters to the set of accounted letters.
        accounted_letters.update(name)
    return selected_hero_names

def get_letter(letters: set[str], available_hero_names: set[str]):
    # Get the hero names that contain the given letter.
    def get_names_containing_letter(letter: str):
        # Get the subset of letters a hero name encompasses from the set of letters.
        def get_encompassed_letters(hero_name: str):
            return letters - (letters - set(hero_name))

        l = [s for s in available_hero_names if letter in s]
        s = set(it.chain.from_iterable(map(get_encompassed_letters, l)))
        return l, s

    # Make a list of tuples of each letter in letters, the list of hero names containing that letter, and the set of encompassed letters.
    l = [(c, *get_names_containing_letter(c)) for c in letters]

    # Sort the list by the letters that are in the fewest names and encompass the most letters.
    l.sort(key=lambda t: (len(t[1]), -len(t[2])))

    # Select the first letter.
    return l[0][0]

def get_names(letter: str, available_hero_names: set[str]):
    return [s for s in available_hero_names if letter in s]

def get_name(l: list[str], letters: set[str]):
    def count_letters(s):
        s = letters - set(s)
        return len(s)
    d = {count_letters(s): s for s in l}
    n = min(d)
    return d[n]

collect_necessary_names(select_names)

8, 84, ANTI-MAGE, CLINKZ, DAWNBREAKER, JAKIRO, NATURE'S PROPHET, NYX ASSASSIN, QUEEN OF PAIN, VOID SPIRIT


This algorithm starts with the unique names. It constructs the set of letters, the accounted set, from those names and loops while there are unaccounted letters, adding the letters of the shortest name that encompasses the most unaccounted letters to the accounted set.

In [5]:
def select_names():
    # Create sets of available and selected hero names from the unique names.
    selected_hero_names = set(unique_names)
    available_hero_names = set(hero_names) - selected_hero_names
    # Create a set of accounted letters.
    accounted_letters = set(it.chain.from_iterable(selected_hero_names))
    nletters = len(alphabet)
    while len(accounted_letters) < nletters:
        # Find the name that encompasses the most unaccounted letters. Select the shortest name in case of ties.
        unaccounted_letters = alphabet - accounted_letters
        l = [(len(unaccounted_letters - set(s)), len(s), s) for s in available_hero_names]
        l.sort()
        s = l[0][-1]
        selected_hero_names.add(s)
        available_hero_names.remove(s)
        accounted_letters.update(s)
    return selected_hero_names
collect_necessary_names(select_names)

8, 77, ANTI-MAGE, CLINKZ, DAWNBREAKER, JAKIRO, NATURE'S PROPHET, NYX ASSASSIN, QUEEN OF PAIN, SVEN


I don't know what I was thinking when I made the first algorithm years ago. The second is much simpler and produces a collection of
the same size. It selects only three different names, two of which are shorter, resulting in fewer total characters.

The first algorithm is sensitive to Python's selection of the value it uses to hash strings. Runs will produce different results.

Interestingly, if I include the longest and shortest names, this algorithm produces a larger collection so the first algorithm is
better in that case. If I exclude them, they produce the same size list, although they contain different members. In both cases,
they don't include the longest and shortest names, which I excluded from the first algorithm.